In [32]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

In [33]:
import os
from datetime import date, timedelta
import pandas as pd
from information.terms_info import tags

pd.set_option('display.max_columns', None)

In [34]:
"""
 Declare data scope
"""
# start_date = date.fromisoformat('2020-01-01')
start_date = date.fromisoformat('2020-02-01')
end_date   = date.fromisoformat('2022-01-30')

end_dates = []
current_date = start_date
while (current_date <= end_date):
    end_dates.append(current_date)
    current_date += timedelta(days=1)
print(f"{len(end_dates)=}")

len(end_dates)=730


In [42]:
for tag in tags:
    normalize_tag = tag.replace('/', '.')
    rsv_directory = os.path.abspath(f'./../../data/external/gt/rsvs/{normalize_tag}')
    msv_df_csv_path = os.path.abspath(f'./../../data/processed/msvs/{normalize_tag}.csv')
    msv_df_pkl_path = os.path.abspath(f'./../../data/processed/msvs/{normalize_tag}.pkl')

    msv_df = None

    for current_date in end_dates:
        current_rsv_path = os.path.join(rsv_directory, f"{current_date}.csv")
        rsv_df = pd.read_csv(current_rsv_path, parse_dates=['date'])
        rsv_df.drop(columns=['isPartial'], inplace=True)
        rsv_df.columns = ['date', 'index']

        if msv_df is None:
            msv_df = rsv_df.copy()
        else:
            # Join data into one
            merged_df = msv_df.merge(rsv_df, on='date', how='outer')

            # Calculate correction factors
            merged_df['correction_factor'] = merged_df['index_y'].values / merged_df['index_x'].values
            merged_df['correction_factor'] = merged_df['correction_factor'].fillna(1)

            aggregated_correction_factor = merged_df['correction_factor'].mean()

            # Resclae the new RSV
            merged_df['index_y'] = merged_df['index_y'] * aggregated_correction_factor

            # Fill new data into previous RSV/MSV
            merged_df['index_x'] = merged_df['index_x'].fillna(merged_df['index_y'])
            print(merged_df.tail())
            
            msv_df = merged_df[['date', 'index_x']].copy()
            msv_df.columns = ['date', 'index']

        print(rsv_df.describe())
        
    msv_df.to_csv(msv_df_csv_path, index=False)
    msv_df.to_pickle(msv_df_pkl_path)

            index
count   30.000000
mean    39.166667
std     26.078242
min     17.000000
25%     22.250000
50%     27.000000
75%     41.000000
max    100.000000
         date     index_x    index_y  correction_factor
26 2020-02-27   96.000000  86.272329           0.947917
27 2020-02-28  100.000000  88.168424           0.930000
28 2020-02-29   89.000000  77.739901           0.921348
29 2020-03-01   83.000000  74.895758           0.951807
30 2020-03-02   94.804757  94.804757           1.000000
         date       index
26 2020-02-27   96.000000
27 2020-02-28  100.000000
28 2020-02-29   89.000000
29 2020-03-01   83.000000
30 2020-03-02   94.804757


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\OneDrive - HKUST\\COVIDNetworkCode\\data\\external\\gt\\rsvs\\.m.07__7\\2020-02-01.csv'